In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_json('/content/drive/MyDrive/ML_Data/term_labeled_texts_train.json')
df.head()

,tokenized_text,token_labels,new_terms
0,"[ABBYY, Retrieval, &, Morphology, Engine, В, с...","[B, I, I, I, E, O, O, O, O, B, E, O, O, O, O, ...","[ABBYY Retrieval & Morphology Engine, программ..."
1,"[Речевые, формулы, в, диалоге, Предложенная, к...","[B, E, O, S, O, O, O, O, O, O, O, B, E, O, O, ...","[Речевые формулы, диалоге, лингвистические тип..."
2,"[Географические, названия, и, полнотекстовые, ...","[B, E, O, B, E, O, B, I, E, O, O, O, O, O, O, ...","[Географические названия, полнотекстовые докум..."
3,"[Методы, автоматического, построения, специали...","[O, B, I, I, E, O, O, O, O, B, I, E, O, O, B, ...",[автоматического построения специализированног...
4,"[К, проблеме, понимания, несегментированного, ...","[O, O, O, B, E, O, O, O, B, E, O, O, O, O, O, ...","[несегментированного текста, метеорологических..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 750 entries, 0 to 749
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tokenized_text  750 non-null    object
 1   token_labels    750 non-null    object
 2   new_terms       750 non-null    object
dtypes: object(3)
memory usage: 23.4+ KB


In [ ]:
texts = df['tokenized_text'].to_list()

In [ ]:
tags = df['token_labels'].to_list()

In [ ]:
print(texts[1])

['Речевые', 'формулы', 'в', 'диалоге', 'Предложенная', 'классификация', ',', 'как', 'и', 'многие', 'другие', 'лингвистические', 'типологии', ',', 'допускает', 'пересечения', '.', 'Например', ',', 'идиома', 'врать', 'готово', 'в', 'последовательности', 'реплик', '[', '-', 'Честное', 'слово', ']', '-', 'Врать', 'готово', ',', 'с', 'одной', 'стороны', ',', 'является', 'комментарием', ',', 'а', 'с', 'другой', '–', 'обладает', 'некоторыми', 'характеристиками', 'формул', 'ответа', ':', 'иллокутивно', 'вынуждается', 'предшествующей', 'репликой', 'и', 'повторяет', 'ее', 'некоторые', 'фонетические', 'особенности', '.', 'Кроме', 'того', ',', 'поскольку', 'оценивается', 'искренность', 'предшествующей', 'клятвы', ',', 'данную', 'идиому', 'можно', 'рассматривать', 'и', 'как', 'формулу', 'эпистемической', 'модальности', '.']


In [ ]:
print(tags[1])

['B', 'E', 'O', 'S', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'E', 'O', 'O', 'O', 'O', 'O', 'O', 'S', 'O', 'O', 'O', 'O', 'S', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'S', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'E', 'O', 'S', 'O', 'O', 'S', 'O', 'O', 'O', 'O', 'B', 'E', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'S', 'O', 'O', 'O', 'O', 'B', 'I', 'E', 'O']


In [ ]:
train_texts = texts[:570]
train_tags = tags[:570]
val_texts = texts[570:]
val_tags = tags[570:]

In [ ]:
unique_tags = ['O', 'S', 'B', 'I', 'E']
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
tag2id

{'O': 0, 'S': 1, 'B': 2, 'I': 3, 'E': 4}

In [ ]:
id2tag

{0: 'O', 1: 'S', 2: 'B', 3: 'I', 4: 'E'}

In [ ]:
! pip install datasets transformers[torch] seqeval accelerate -U

In [ ]:
model_checkpoint = "distilbert-base-uncased"
batch_size = 4

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
encoded_tags_train = [[tag2id[tag] for tag in doc] for doc in train_tags]

In [ ]:
encoded_tags_val = [[tag2id[tag] for tag in doc] for doc in val_tags]

In [ ]:
label_all_tokens = True

def tokenize_and_align_labels(texts, tags):

    tokenized_inputs = tokenizer(texts,
                                 #max_length=1150, не работает для Bert
                                 truncation=True, # обрезка слишком длинных последовательностей
                                 padding=True,
                                 is_split_into_words=True # предупреждаем, что вход поступит в виде списков токенов
                                 )

    labels = []
    for i, label in enumerate(tags):

        # достаем 1 текст
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []

        # идем по всем словам
        for word_idx in word_ids:

            # Некоторые специальные токены имеют id None. Мы даем им лейбл -100, чтобы модель их игнорировала
            if word_idx is None:
                label_ids.append(-100)

            # Логично, что если слово разделилось на subword-токены, их лейблы в пределах слова должны быть одинаковыми.
            # Если мы перешли на новое слово, добавляем его лейбл в список
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])

            # Всем следующим частям одного и того же слова мы даем или тот же лейбл, или -100, если label_all_tokens=False
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)

            previous_word_idx = word_idx

        labels.append(label_ids)

    return tokenized_inputs, labels

In [ ]:
tokenized_train, train_labels = tokenize_and_align_labels(train_texts, encoded_tags_train)

In [ ]:
print(tokenized_train[2])

Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


In [ ]:
print(train_labels[2])

[-100, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 2, 2, 2, 2, 2, 3, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 2, 2, 2, 2, 2, 3, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
len(train_labels[2])

512

In [ ]:
tokenized_val, val_labels = tokenize_and_align_labels(val_texts, encoded_tags_val)

In [ ]:
len(val_labels[12])

512

In [ ]:
len(train_labels[67])

512

In [ ]:
import torch

In [ ]:
class TermLabDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
      # этот метод вызывается моделью, когда она учится
      # он определяет, в каком виде данные подаются в модель
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = TermLabDataset(tokenized_train, train_labels)
val_dataset = TermLabDataset(tokenized_val, val_labels)

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=5)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"//content/sample_data/{model_name}-finetuned-1-pos-chu",
    overwrite_output_dir=True, # записываем каждый раз в один и тот же файл
    evaluation_strategy = "epoch", # оцениваем каждую эпоху
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size, # размер тренировочного батча на каждый процессор
    per_device_eval_batch_size=batch_size,
    num_train_epochs=50,
    weight_decay=0.01, # регуляризация функции потерь
    push_to_hub=False, # не публиковать на Huggingface
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
from datasets import load_metric

In [ ]:
metric = load_metric("seqeval")

<ipython-input-33-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


The repository for seqeval contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/seqeval.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Удаляем индексы специальных токенов
    true_predictions = [
        [id2tag[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2tag[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.800970,0.609607,0.564803,0.586351,0.696092
2,No log,0.733369,0.679001,0.528159,0.594155,0.729304
3,No log,0.701183,0.630034,0.760287,0.689059,0.734887
4,0.829100,0.668398,0.637132,0.786021,0.703788,0.743871
5,0.829100,0.642953,0.674997,0.701649,0.688065,0.752832
6,0.829100,0.611803,0.696713,0.732717,0.714262,0.771454
7,0.534800,0.639635,0.705576,0.732751,0.718907,0.777646
8,0.534800,0.768247,0.616096,0.851378,0.714876,0.731671
9,0.534800,0.670107,0.683897,0.782246,0.729773,0.772281
10,0.534800,0.652758,0.707750,0.761360,0.733577,0.783574


TrainOutput(global_step=7150, training_loss=0.17937579878560314, metrics={'train_runtime': 1854.6694, 'train_samples_per_second': 15.367, 'train_steps_per_second': 3.855, 'total_flos': 3723815333376000.0, 'train_loss': 0.17937579878560314, 'epoch': 50.0})

In [ ]:
predictions, labels, _ = trainer.predict(val_dataset)
predictions = np.argmax(predictions, axis=2)

# Уберем игнорируемые токены и декодируем предсказанные токены
ids_predictions = [
    [id2tag[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [id2tag[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=ids_predictions, references=true_labels)
results

{'_': {'precision': 0.7266049884353316,
  'recall': 0.8051745635910225,
  'f1': 0.7638747412348438,
  'number': 28872},
 'overall_precision': 0.7266049884353316,
 'overall_recall': 0.8051745635910225,
 'overall_f1': 0.7638747412348438,
 'overall_accuracy': 0.8007030765342462}

In [ ]:
from transformers import TokenClassificationPipeline

In [ ]:
def postprocess_output(output):

  string = ''
  tags = []
  last_token_end = 0

  for token in output:
    # сначала обработаем pos-тег
    if '_' in token['entity']:
      pos = id2tag[int(token['entity'].split('_')[1])]
    else:
      pos = token['entity']

    if token['word'][0] != '#':
      # если токен - или начало слова, или полное слово
      if last_token_end != token['start']:
        string += ' '

      string += token['word']
      tags.append(pos)

    else:
      # если токен - середина слова или конец
      string += token['word'].replace('#', '')

    last_token_end = token['end']

  return list(zip(string.split(), tags))

In [ ]:
checkpoint = '/content/sample_data/distilbert-base-uncased-finetuned-1-pos-chu/checkpoint-2500'

In [ ]:
tokenizer_term = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
model_term = AutoModelForTokenClassification.from_pretrained(checkpoint)

In [ ]:
termbert = TokenClassificationPipeline(model=model_term, tokenizer=tokenizer_term, task="pos")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
postprocess_output(termbert('В данной статье описываются основные принципы работы синтаксического парсера русского языка LPaRus , разработанного на основе лингвистических технологий компании Megaputer Intelligence .'))

[('в', 'O'),
 ('даннои', 'O'),
 ('статье', 'O'),
 ('описываются', 'O'),
 ('основные', 'O'),
 ('принципы', 'O'),
 ('работы', 'O'),
 ('синтаксического', 'B'),
 ('парсера', 'E'),
 ('русского', 'B'),
 ('языка', 'E'),
 ('lparus', 'S'),
 (',', 'O'),
 ('разработанного', 'O'),
 ('на', 'O'),
 ('основе', 'O'),
 ('лингвистических', 'B'),
 ('технологии', 'E'),
 ('компании', 'E'),
 ('megaputer', 'S'),
 ('intelligence', 'S'),
 ('.', 'O')]

In [ ]:
checkpoint = '/content/sample_data/distilbert-base-uncased-finetuned-1-pos-chu/checkpoint-4000'

In [ ]:
tokenizer_term = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
model_term = AutoModelForTokenClassification.from_pretrained(checkpoint)

In [ ]:
termbert = TokenClassificationPipeline(model=model_term, tokenizer=tokenizer_term, task="pos")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
postprocess_output(termbert('В данной статье описываются основные принципы работы синтаксического парсера русского языка LPaRus , разработанного на основе лингвистических технологий компании Megaputer Intelligence .'))

[('в', 'O'),
 ('даннои', 'O'),
 ('статье', 'O'),
 ('описываются', 'O'),
 ('основные', 'O'),
 ('принципы', 'O'),
 ('работы', 'O'),
 ('синтаксического', 'B'),
 ('парсера', 'E'),
 ('русского', 'B'),
 ('языка', 'E'),
 ('lparus', 'S'),
 (',', 'O'),
 ('разработанного', 'O'),
 ('на', 'O'),
 ('основе', 'O'),
 ('лингвистических', 'B'),
 ('технологии', 'E'),
 ('компании', 'S'),
 ('megaputer', 'S'),
 ('intelligence', 'E'),
 ('.', 'O')]

In [ ]:
checkpoint = '/content/sample_data/distilbert-base-uncased-finetuned-1-pos-chu/checkpoint-7150'

In [ ]:
tokenizer_term = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
model_term = AutoModelForTokenClassification.from_pretrained(checkpoint)

In [ ]:
termbert = TokenClassificationPipeline(model=model_term, tokenizer=tokenizer_term, task="pos")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
postprocess_output(termbert('В данной статье описываются основные принципы работы синтаксического парсера русского языка LPaRus , разработанного на основе лингвистических технологий компании Megaputer Intelligence .'))

[('в', 'O'),
 ('даннои', 'O'),
 ('статье', 'O'),
 ('описываются', 'O'),
 ('основные', 'O'),
 ('принципы', 'O'),
 ('работы', 'O'),
 ('синтаксического', 'B'),
 ('парсера', 'E'),
 ('русского', 'B'),
 ('языка', 'E'),
 ('lparus', 'S'),
 (',', 'O'),
 ('разработанного', 'O'),
 ('на', 'O'),
 ('основе', 'O'),
 ('лингвистических', 'B'),
 ('технологии', 'E'),
 ('компании', 'O'),
 ('megaputer', 'S'),
 ('intelligence', 'E'),
 ('.', 'O')]